In [1]:
from consensus.PBFT_Simulator import PBFT_Simulator
from consensus.QPBFT_Simulator import QPBFT_Simulator
from consensus.Proposed_Simulator import Proposed_Simulator
import time

In [2]:
consensus = PBFT_Simulator(10, 2)

In [3]:
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
delay = end-start
print(f"Spend time => {delay} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Faulty => False, Send_messages => ('Hello world 0', 'pre-prepare', 0), Receive_messages => [('Hello world 0', 'pre-prepare', 0, 'e8f78999cb260a9af5eb9d9808d13151bc0fc7530304620e7507f8e87f1ff521')]
IdUser => 1, Faulty => True , Send_messages => ''                                 , Receive_messages => []
IdUser => 2, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 3, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 4, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 5, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 6, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 7, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 8, Faulty => True

In [4]:
del consensus

In [5]:
consensus = QPBFT_Simulator(6, 4, 2)

In [6]:
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
spend = end-start
print(f"Spend time => {end - start} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0,Score => 0.77, Role =>  Manager, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepare', 0)    , Receive_messages_log => [('Hello world 0', 'prepare', 0, '5310eca6643a80e0bafcc136e6c89b61921827e21289714b66752fb1d9425448')]
IdUser => 2,Score => 0.68, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 3,Score => 0.71, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 4,Score => 0.76, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 5,Score => 1.00, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
------------------------------
Prepare Phase
IdUser => 0,Score => 0.77, Role =>  Manager, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepa

In [7]:
del consensus

In [8]:
consensus = Proposed_Simulator(6, 4, 5, 3)
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 6, Priority => 0.74, Role =>   Slave, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepare', 6)    , Receive_Messages_Log => [('Hello world 0', 'prepare', 6, 'ea9b28ff49b0961bc2d8652f13862064c4cb3bda24c10a8e4965202fb6e4a671')]
IdUser => 5, Priority => 0.62, Role =>  Master, Faulty =>  False, Send_message_log => ''                                 , Receive_Messages_Log => []
IdUser => 9, Priority => 0.48, Role =>   Slave, Faulty =>  False, Send_message_log => ''                                 , Receive_Messages_Log => []
IdUser => 1, Priority => 0.26, Role =>  Master, Faulty =>   True, Send_message_log => ''                                 , Receive_Messages_Log => []
IdUser => 0, Priority => 0.00, Role =>  Master, Faulty =>  False, Send_message_log => ''                                 , Receive_Messages_Log => []
------------------------------
Prepare Phase
IdUser => 6, Priority => 0.74, Role =>   Slave, Faulty =>  False, Send_message_log => ('Hell

In [9]:
del consensus